In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import time, os, shutil, re
import requests

In [17]:
# investing.com 이게 크롤링 돌아가는 코드
main_path = 'C:/KDT4/학원/investing_crawl'


main = 'https://www.investing.com'
topics = ['stock-market-news', 'commodities-news', 'forex-news', 'economy', 'economic-indicators','politics','world-news']

for topic in topics[4:5]:
    title_lst = []
    content_lst = []
    date_lst = []
    company_lst = []
    save_path = '/'.join([main_path, topic])
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    url = '/'.join([main,'news', topic])
    headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
            'Referer': '/'.join([main, 'news', topic])}
    res = requests.get(url,headers=headers)
    time.sleep(0.1)
    print(topic, int(BeautifulSoup(res.text, 'html.parser').select('#paginationWrap > div.midDiv.inlineblock > a:nth-child(5)')[0]['title'].split()[-1])//17)
    for page in range(1001, int(BeautifulSoup(res.text, 'html.parser').select('#paginationWrap > div.midDiv.inlineblock > a:nth-child(5)')[0]['title'].split()[-1])//17):
        print(page)
        url = '/'.join([main,'news', topic, str(page)])
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
            'Referer': '/'.join([main, 'news', topic, str(page)])}
        res = requests.get(url,headers=headers)
        time.sleep(0.1)
        soup = BeautifulSoup(res.text, 'html.parser')
        article_path = '#leftColumn > div.largeTitle > article'
        nlstdir = soup.select(article_path)
        if not nlstdir:
            print('done')
            break
        for newslst in nlstdir:
            title = np.nan
            contents = np.nan
            date = np.nan
            try:
                if 'dfp-native' in newslst['class']:
                    continue
                elif 'https://' in newslst.select('a')[0]['href']:
                    newsurl = newslst.select('a')[0]['href']
                else : 
                    newsurl = main+newslst.select('a')[0]['href']
                newsres = requests.get(newsurl, headers = headers)
                time.sleep(0.1)
                news = BeautifulSoup(newsres.text,'html.parser')
                if newslst.select('div.textDiv > span > span:nth-child(1)'):
                    company = newslst.select('div.textDiv > span > span:nth-child(1)')[0].text
                    if company == 'By Investing.com Studios':
                        continue
                    title = news.select('#leftColumn > h1')[0].text
                    date = news.select('#leftColumn > div.contentSectionDetails > span')[0]
                    contents = []
                    for con in news.select('#leftColumn > div.articlePage > p'):
                        contents.append(con.text)
                    contents = '\n'.join(contents)

                elif newslst.select('div.textDiv > div > span:nth-child(1)'):
                    company = newslst.select('div.textDiv > div > span:nth-child(1)')[0].text
                    # 각 회사별 크롤링 돌아가게하는 코드 작성해야함
                    if company == 'By CNBC': # Sample#1 : CNBC
                        title = news.select('#main-article-header > div:nth-child(1)>div:nth-child(1)>div:nth-child(2)')[0].text
                        date = news.select('#main-article-header > div:nth-child(1)>div:nth-child(1)>div:nth-child(3)>time')[0]['datetime']
                        contents = []
                        for con in news.select('#RegularArticle-ArticleBody-5 > div.group'):
                            contents.append(con.text)
                        contents = '\n'.join(contents)
                    # CNBC 랑 비슷하게 contents title date 받을수 있게 코드좀 만들어주세요
                    # 각 회사별 링크는 직접 찾아서 들어가봐야합니다.. 아래가 종류
                    elif company == 'By CryptoDaily': # disclaimer 후처리해야함
                        title = news.select('#PostsPage > div.bg-body.box-layout > h2')[0].text
                        date = news.select('#PostsPage > div.bg-body.box-layout > div.row.article-meta > div > div > div > div.date-count')[0].text
                        contents = []
                        for lst in news.select('div.news-content'):
                            contents.append(lst.text)
                        contents = '\n'.join(contents)
                    elif company == 'By CryptoGlobe':
                        title = news.select('#content_start > div:nth-child(1) > article > header > div:nth-child(1) > h1')[0].text
                        date = news.select('#content_start > div:nth-child(1) > article > header > div.row > div:nth-child(1) > div > div:nth-child(2) > ul > li:nth-child(1)')[0].text
                        contents = news.select('div.article-body')[0].text
                    elif company == 'By Huffington Post':
                        title = news.select('#entry-header > header > div.top-header > h1')[0].text
                        date = news.select('#entry-header > header > div.bottom-header > div.bottom-header__left > div.timestamp > time:nth-child(1) > span')[0].text
                        divs = news.select('#entry-body > section')[0].text
                        contents = []
                        for i in divs.split('\n'):
                            contents.append(i.strip('Advertisement'))
                        contents = '\n'.join(contents)
                    else:
                        pass
                else:
                    print(f'Error Occured in {topic} page:{page}')
                    pass
            except:
                print(f'Error Occured in {topic} page:{page}')
                continue
            company_lst.append(company)
            title_lst.append(title)
            date_lst.append(date)
            content_lst.append(contents)
        if page%100 == 0:
            pd.DataFrame({'company':company_lst, 'date':date_lst, 'title':title_lst, 'content':content_lst}).to_csv(save_path+f'/news_{topic}_{page//100:05}.csv')
            title_lst = []
            content_lst = []
            date_lst = []
            company_lst = []
    pd.DataFrame({'company':company_lst, 'date':date_lst, 'title':title_lst, 'content':content_lst}).to_csv(save_path+f'/news_{topic}_{page//100+1:05}.csv')

economic-indicators 1880
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195


In [111]:
newslst['class']

['articleItem', 'dfp-native']

In [116]:
pd.read_csv('D:/Project4/data/investing_crawl/stock-market-news/news_stock-market-news_00001.csv',index_col=0).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   company  103 non-null    object
 1   date     99 non-null     object
 2   title    99 non-null     object
 3   content  98 non-null     object
dtypes: object(4)
memory usage: 4.0+ KB


In [88]:
news = BeautifulSoup(requests.get("https://invst.ly/xxtbk").text, 'html.parser')
news.select('#main-article-header > div > div.ArticleHeader-wrapperHeroNoImage.ArticleHeader-wrapperHero.ArticleHeader-wrapper.ArticleHeader-wrapperNoImage > div.ArticleHeader-time > time')

[]

In [96]:
print(news.select('#content_start > div:nth-child(1) > article > header > div:nth-child(1) > h1')[0].text)
print(news.select('#content_start > div:nth-child(1) > article > header > div.row > div:nth-child(1) > div > div:nth-child(2) > ul > li:nth-child(1)')[0].text)
for lst in news.select('div.article-body'):
    continue
print(lst.text)

$USDC: World’s Largest Asset Manager With $9.5 Trillion AUM Interested in USD Coin
16 Apr 2022

NYC-based BlackRock Inc., which is the world’s top asset management firm with around $9.5 trillion in total assets under management (AUM) as of 30 September 2021 according to ADV Ratings, has invested in stablecoin issuer Circle Internet Financial Ltd.
On 26 September 2018, Circle announced that 26 September 2018 announced that it was launching its “USD Coin” ($USDC for short), which is a regulated fully collateralized dollar-backed cryptocurrency that was originally announced on 16 May 2018. This stablecoin is based on an open source fiat stablecoin framework developed and governed by the CENTRE project. Circle said back then that “individuals and institutions” could enroll its dollar tokenization service to “deposit US dollars from bank accounts, convert those dollars into tokens usable everywhere the internet reaches (subject to the token’s compliance controls), and redeem USDC tokens and

In [97]:
t = 100
f'{t:05}'

'00100'

In [6]:
int(BeautifulSoup(res.text, 'html.parser').select('#paginationWrap > div.midDiv.inlineblock > a:nth-child(5)')[0]['title'].split()[-1]//17)

IndexError: list index out of range

In [9]:
int(BeautifulSoup(res.text, 'html.parser').select('#paginationWrap > div.midDiv.inlineblock > a:nth-child(5)')[0]['title'].split()[-1])//17

IndexError: list index out of range

In [10]:
res

<Response [404]>

In [11]:
res.text

' <!DOCTYPE HTML>\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" xmlns:schema="http://schema.org/" class="com" lang="en" geo="KR">\n<head>\n<link rel="dns-prefetch" href="https://i-invdn-com.investing.com" />\n<link rel="dns-prefetch" href="https://a-invdn-com.investing.com" />\n<script>\n        window.dataLayerGTMParams = window.dataLayerGTMParams || [];\n        dataLayerGTMParams.push({"client_name":"legacy-desktop"});\n    </script>\n\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n                new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n            j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n            \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n        })(window,document,\'script\',\'dataLayerGTMParams\',\'GTM-PGT3R3D\');</script>\n\n<script type="text/javascript">\n    function OptanonWrapper() { }\n\n    // temp for checks\

'https://www.investing.com/news/e'